# Example 5 -- Using the API to Dynamically Adjust River Conductance

## Create a Simple Model

In [ ]:
import flopy
import flopy.mf6 as fp
libmf6 = "/Users/langevin/langevin/dev/modflow6-fork.git/bin/libmf6.dylib"

name = "mymodel"
ws = "./ex5"
sim = fp.MFSimulation(sim_name=name, sim_ws=ws, memory_print_option="all")
pd = [(1, 1, 1.), (50., 50, 1.)]
tdis = fp.ModflowTdis(sim, nper=len(pd), perioddata=pd)
ims = fp.ModflowIms(sim, complexity="simple")
gwf = fp.ModflowGwf(
    sim,
    modelname=name, 
    print_input=True, 
    save_flows=True,
)
dis = fp.ModflowGwfdis(
    gwf, 
    nlay=3, 
    nrow=21, 
    ncol=20, 
    delr=500., 
    delc=500.,
    top=360, 
    botm=[220, 200, 0],
)
npf = fp.ModflowGwfnpf(
    gwf, 
    save_specific_discharge=True,
    k=[50., .01, 200.], 
    k33=[10., 0.01, 20.]
)
sto = fp.ModflowGwfsto(
    gwf,
    sy=0.1,
    ss=1.e-5,
    iconvert=[1, 0, 0],
    steady_state={0:True, 1:False}, 
    transient={0:False, 1:True}
)
ic = fp.ModflowGwfic(gwf, strt=330.)
condref = 100000.
spd = [(0, i, 19, 320., condref, 317.) for i in range(21)]
riv = fp.ModflowGwfriv(gwf, stress_period_data=spd, pname="RIVER", print_flows=True)
rch = fp.ModflowGwfrcha(gwf, recharge=0.005, pname="RECHARGE")
well_loc = [0, 11, 16]
spd = {0:[well_loc + [0.]], 1:[well_loc + [-300000.]]}
wel = fp.ModflowGwfwel(gwf, stress_period_data=spd, pname="WELL")
oc = fp.ModflowGwfoc(
    gwf,
    headprintrecord=[("COLUMNS", 20, "WIDTH", 15, "DIGITS", 6, "GENERAL")],
    head_filerecord=f"{name}.hds",
    budget_filerecord=f"{name}.cbc",
    printrecord=[("head", "all"), ("budget", "all")],
    saverecord=[("head", "all"), ("budget", "all")],
)
sim.write_simulation()
sim.run_simulation()

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation
head_all = gwf.output.head().get_alldata()
head = head_all[-1]
h_well = head_all[:, well_loc[0], well_loc[1], well_loc[2]]
h_riv = head_all[:, well_loc[0], well_loc[1], -1]
bud = gwf.output.budget()
times = bud.times
spdis = bud.get_data(text='DATA-SPDIS')[0]
qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(spdis, gwf)

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.set_aspect(1)
ax.set_xlabel(r'x')
ax.set_ylabel(r'y')
title = ax.set_title(f"Time = {times[0]} days")

# plot time zero
pmv = flopy.plot.PlotMapView(gwf, ax=ax, layer=0)
pmv.plot_grid(linewidth=0.1)
pmv.plot_bc(name="river")
pmv.plot_bc(name="well")
levels = np.arange(int(head_all.min()), int(head_all.max()) + 4, 2)
cmap = plt.colormaps["jet"].with_extremes(under="white", over="white")
ca_dict = {
    "cmap": cmap, 
    "extend": "both", 
    "levels":levels, 
    "filled":True,
    "norm": "linear",
}
cont = pmv.contour_array(head_all[0], **ca_dict)
fig.colorbar(cont, shrink=0.5)

def animate(i):
    global cont
    global title
    for c in cont.collections:
        c.remove()  # removes only the contours, leaves the rest intact
    cont = pmv.contour_array(head_all[i], **ca_dict)
    title = ax.set_title(f"Time = {times[i]:.2f} days")
    return cont

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=head_all.shape[0])
plt.close()

from IPython.display import HTML
HTML(ani.to_jshtml())

## Use API to Run Model

In [ ]:
import modflowapi

verbose = False
mf6 =  modflowapi.ModflowApi(libmf6, working_directory=ws)
if verbose: print("Initializing mf6...")
mf6.initialize()

# get a pointer to the head variable
head_address = mf6.get_var_address("X", "SLN_1")
head = mf6.get_value_ptr(head_address)

istep = 0
current_time = 0.
end_time = mf6.get_end_time()

while current_time < end_time:
    
    if verbose: print(f"  Get and prepare time step {istep}...")
    dt = mf6.get_time_step()
    mf6.prepare_time_step(dt)
    
    kiter = 0
    if verbose: print("  Prepare solve...")
    mf6.prepare_solve(1)
    while kiter < 30:
        if verbose: print("    Solve...")
        has_converged = mf6.solve(1)
        if verbose: print (f"    Head min ({head.min()}) max ({head.max()})")
        if has_converged:
            break
    if verbose: print("  Finalize solve...")
    mf6.finalize_solve(1)
    
    if verbose: print("  Finalize time step...")
    mf6.finalize_time_step()
    current_time = mf6.get_current_time()
    istep += 1

if verbose: print ("Finalizing mf6...")
mf6.finalize()
print("done...")

## Run API Model with Modified River Conductance

In [ ]:
import modflowapi

verbose = False
mf6 =  modflowapi.ModflowApi(libmf6, working_directory=ws)
if verbose: print("Initializing mf6...")
mf6.initialize()

# construct variable addresses
head_address = mf6.get_var_address("X", "SLN_1")
nodelist_address = mf6.get_var_address("NODELIST", name.upper(), "RIVER")
river_bound_address = mf6.get_var_address("BOUND", name.upper(), "RIVER")

# retrieve pointers to variables within MODFLOW
head = mf6.get_value_ptr(head_address)
nodelist = mf6.get_value_ptr(nodelist_address)
river_bound = mf6.get_value_ptr(river_bound_address)

current_time = 0.
end_time = mf6.get_end_time()

while current_time < end_time:
    
    if verbose: print("  Get and prepare time step...")
    dt = mf6.get_time_step()
    mf6.prepare_time_step(dt)
    
    kiter = 0
    if verbose: print("  Prepare solve...")
    mf6.prepare_solve(1)
    while kiter < 30:
        
        for i in range(nodelist.shape[0]):
            icell = nodelist[i] - 1
            if head[icell] > 320.:
                river_bound[i, 1] = condref
            else:
                river_bound[i, 1] = condref * 0.8
            if verbose: print(f"riv {i} cond {river_bound[i, 1]} sim head {head[icell]}")
        
        if verbose: print("    Solve...")
        has_converged = mf6.solve(1)
        if verbose: print (f"    Head min ({head.min()}) max ({head.max()})")
        if has_converged:
            break
    if verbose: print("  Finalize solve...")
    mf6.finalize_solve(1)
    
    if verbose: print("  Finalize time step...")
    mf6.finalize_time_step()
    current_time = mf6.get_current_time()

if verbose: print ("Finalizing mf6...")
mf6.finalize()
print("done...")

In [ ]:
head_all = gwf.output.head().get_alldata()
h_well_api = head_all[:, well_loc[0], well_loc[1], well_loc[2]]
h_riv_api = head_all[:, well_loc[0], well_loc[1], -1]
bud = gwf.output.budget()
times = bud.times

ax = plt.subplot(1, 1, 1)
plt.plot(times, h_well, "bo", mfc="none", label="Head at well (reference)")
plt.plot(times, h_well_api, "bx", label="Head at well (api)")
plt.plot(times, h_riv, "go", mfc="none", label="Head at river (reference)")
plt.plot(times, h_riv_api, "gx", label="Head at river (api)")
ax.set_xlabel("time, in days")
ax.set_ylabel("head, in meters")
plt.legend(loc="upper center")

In [ ]:
ax = plt.subplot(1, 1, 1)
ax.set_xlabel("time, in days")
ax.set_ylabel("Head difference (reference - api), in meters")
ax.plot(times, h_well - h_well_api, "b-", label="Head at well")
ax.plot(times, h_riv - h_riv_api, "g-", label="Head at river")
plt.legend(loc="upper center")